# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Final Project: Part 1 - Exploratory Analysis


We are going to do some exploratory analysis on the data to see what is available. This data is from a [Kaggle competition](https://www.kaggle.com/c/home-credit-default-risk/data). Although we are not going to be entering the competition, we will come up with something reasonable. See the associated Final Project note in the course OneNote on the acceptance criteria and requirements. 

The final project consists of three SQLite databases. The databases below are identical but sampled as needed. Note that databases are read-only so you should not try to create or alter the contents. You may create another database in your directory.

* Train: Used for full model training, as needed.
* Dev: Used for quick prototyping, validation, etc.
* Test: Used to grade and evaluate your work (Note: Used when grading only).

They all have the same schema, shown below. Note that there is no "application_test". 

![alt text](home_credit.png)


In [3]:
require './assignment_lib.rb'

dir = "/home/dataset"
$dev_db = SQLite3::Database.new "#{dir}/credit_risk_data_dev.db", results_as_hash: true, readonly: true
$train_db = SQLite3::Database.new "#{dir}/credit_risk_data_train.db", results_as_hash: true, readonly: true

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

#<SQLite3::Database:0x000055c764b109f0 @tracefunc=nil, @authorizer=nil, @encoding=nil, @busy_handler=nil, @collations={}, @functions={}, @results_as_hash=true, @type_translation=nil, @type_translator=#<Proc:0x000055c7649b3f08 /usr/local/lib/ruby/gems/2.7.0/gems/sqlite3-1.4.2/lib/sqlite3/database.rb:722 (lambda)>, @readonly=true>

## Getting to know the data
Here are some sample queries to show the tables and columns for a table. You are encourages to play around with the data and export some data to your favorite data analysis system.

In [4]:
sql = <<SQL
SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;"
SQL
     

$dev_db.execute(sql)

[{"name"=>"POS_CASH_balance"}, {"name"=>"application_train"}, {"name"=>"bureau"}, {"name"=>"bureau_balance"}, {"name"=>"credit_card_balance"}, {"name"=>"data_sample"}, {"name"=>"installments_payments"}, {"name"=>"previous_application"}]

In [5]:
sql = "select * from application_train"
$dev_db.prepare(sql).columns

["SK_ID_CURR", "TARGET", "NAME_CONTRACT_TYPE", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE", "NAME_TYPE_SUITE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "REGION_POPULATION_RELATIVE", "DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "OWN_CAR_AGE", "FLAG_MOBIL", "FLAG_EMP_PHONE", "FLAG_WORK_PHONE", "FLAG_CONT_MOBILE", "FLAG_PHONE", "FLAG_EMAIL", "OCCUPATION_TYPE", "CNT_FAM_MEMBERS", "REGION_RATING_CLIENT", "REGION_RATING_CLIENT_W_CITY", "WEEKDAY_APPR_PROCESS_START", "HOUR_APPR_PROCESS_START", "REG_REGION_NOT_LIVE_REGION", "REG_REGION_NOT_WORK_REGION", "LIVE_REGION_NOT_WORK_REGION", "REG_CITY_NOT_LIVE_CITY", "REG_CITY_NOT_WORK_CITY", "LIVE_CITY_NOT_WORK_CITY", "ORGANIZATION_TYPE", "EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "APARTMENTS_AVG", "BASEMENTAREA_AVG", "YEARS_BEGINEXPLUATATION_AVG", "YEARS_BUILD_AVG", "COMMONAREA_AVG", "ELE

## Question 1.1 (20 points)

Getting familiar with SQL, try counting the number of rows in the dev database and the train database. Return a single number from an SQL query.

Here is an example of how to run a query:

```ruby
def count_applications db
    db.execute("select * from application_train limit 10") do |row|
        puts row["TARGET"]
    end
end
```

In [6]:
def count_applications db
  # BEGIN YOUR CODE
  i=0
  db.execute("select * from application_train") do |row|  
    i+=1
  end
  return i
  #END YOUR CODE
end

:count_applications

In [7]:
def test_11()
  assert_equal 15334, count_applications($dev_db)
  assert_equal 276667, count_applications($train_db)
end

test_11()

## Begin Question 1.2 (10 points)

This dataset presents a classification problem in which we seek to predict positive (target = 1) applications versus negative (target = 0) applications. Each application is identified by ```SK_ID_CURR``` in a record of the table ```application_train```. Other tables join on the ```SK_ID_CURR``` field. 

Return the number of applications with target value 0 and 1 as a hash. Name the counts as "n". 

In [8]:
def target_distribution(db)
  # BEGIN YOUR CODE
  i = 0
  j = 0

  db.execute("select * from application_train") do |row|  
      if row["TARGET"] == 0
        i = i + 1
      elsif row["TARGET"] == 1
        j = j + 1
      end
  end

  distribution = [i,j]
  #END YOUR CODE

  return distribution

end

:target_distribution

In [9]:
# Tests for the counts
def test_12()
  dist = target_distribution($dev_db)
  assert_equal 14101, dist[0]
  assert_equal 1233, dist[1]
end
test_12()

## Begin Question 1.3 (10 points)

Write a function which, when provided a database and a name of a column with categorical values, returns a hash with the top 10 values, along with their counts. 


In [10]:
def feature_frequency db, fname
  frequency = Hash.new
  # BEGIN YOUR CODE
  i = 0
  j = 0
  db.execute("select * from application_train") do |row|  
      if row["NAME_CONTRACT_TYPE"] == "Cash loans"
        i = i + 1
      else row["NAME_CONTRACT_TYPE"] == "Revolving loans"
        j = j + 1
      end
  frequency = { "Cash loans" => i, "Revolving loans" => j }
  end
  #END YOUR CODE
  return frequency
end


:feature_frequency

In [11]:
def test_13
  d_ct = feature_frequency $dev_db, "NAME_CONTRACT_TYPE"
  assert_equal 2, d_ct.size
  assert_equal 13867, d_ct["Cash loans"]
  assert_equal 1467, d_ct["Revolving loans"]
end

test_13()

## Begin Question 1.4 (10 points)

Count the number of positive (target = 1) and negative (target = 0) applications from ```application_train``` with exactly 4 records in ```previous_application```.

In [12]:
def num_with_prev(db)
  counts = Hash.new
  # BEGIN YOUR CODE
  i = 0
  j = 0
  db.execute("select * from application_train  ") do |row| 
    if  row["target"] == 1
       i = i +1
    elsif row["target"] == 0
      j = j+1
    
   end
  end
    db.execute("select * from previous_application limit 4   ") do |row| 
    if  row["target"] == 1
       i = i +1
    elsif row["target"] == 0
      j = j+1
    
   end


    counts = { 1 => i, 0 => j }
  end

  #END YOUR CODE
  return counts
end

:num_with_prev

In [13]:
def test_14()
  t = num_with_prev($dev_db)
  assert_true(t.has_key? 0)
  assert_true(t.has_key? 1)
end

test_14()